# Import

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime

# Constant

In [3]:
X_TRAIN_PATH = 'data/digit_xtrain.csv'
X_TEST_PATH = 'data/digit_xtest.csv'
Y_TRAIN_PATH = 'data/digit_ytrain.csv'
Y_TEST_PATH = 'data/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NUMBER_CLASSES = 10
VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT * IMAGE_WIDTH * CHANNELS

# Gather Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

Wall time: 26.8 s


# Data preprocessing

In [5]:
# Re-scale to 0-1
x_train_all, x_test = x_train_all/255.0, x_test/255.0

## Conver target values to one-hot encoding

In [6]:
y_train_all = np.eye(NUMBER_CLASSES)[y_train_all]
print(y_train_all.shape)
y_test = np.eye(NUMBER_CLASSES)[y_test]
print(y_test.shape)

print(x_test.shape)

(60000, 10)
(10000, 10)
(10000, 784)


In [7]:
x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [51]:
# rerun all below
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')  
# we leave the first dimension black, first dim is how many example we are going to use
# this will actually be determined a little later on, when training model, we will split data to batch
Y = tf.placeholder(tf.float32, shape=[None, NUMBER_CLASSES], name='labels')

## Neural Network Architecture

In [52]:
epoch_number = 10
learning_rate = 0.0001  # 1e-4

hidden_1_number = 512
hidden_2_number = 64

In [53]:
def setup_layer(input_before, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='w')
        
        initial_bias = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_bias, name='b')

        layer_input = tf.matmul(input_before, w) + b
        if name == 'out':
            layer_output = tf.nn.softmax(layer_input)
        else:
            layer_output = tf.nn.relu(layer_input)
        return layer_output

In [54]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, hidden_1_number],
                     bias_dim=[hidden_1_number],
                     name='layer_1')
layer_2 = setup_layer(layer_1, weight_dim=[hidden_1_number, hidden_2_number],
                     bias_dim=[hidden_2_number],
                     name='layer_2')
output = setup_layer(layer_2, weight_dim=[hidden_2_number, NUMBER_CLASSES],
                     bias_dim=[NUMBER_CLASSES],
                     name='output')

# Tensorboard setup

In [55]:
name = 'model'
folder_name = f'Model {name} at {strftime("%D %H%M")}'.replace('/', ';')
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as oserr:
    print('error :', oserr.strerror)
else:
    print('Successfully create file')

Successfully create file


# Loss, Optimisation and Metric

In [56]:
with tf.name_scope('loss'): 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))
with tf.name_scope('optimizer'):    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss)

## Accuracy Metric

In [57]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [58]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)

#### Check Input Images in Tensorboard

In [59]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])  # reshape to picture shape
    tf.summary.image('image_input', x_image, max_outputs=4)  # only 4 image willl show

# Run session

In [60]:
session = tf.Session()
init = tf.global_variables_initializer()  # initialise all the variables
session.run(init)

### Setup Filewriter and Merge Summaries

In [61]:
merged_summary = tf.summary.merge_all()  # all the calcalation whta we want to read, we will combine and merge them and store it to this var
train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(session.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

# Batching the Data

In [62]:
size_of_batch = 1000
examples_number = y_train.shape[0]
iteration_number = int(examples_number/size_of_batch)

epoch_index = 0

In [63]:
def get_batch(size_of_batch, data, labels):
    global examples_number
    global epoch_index
    
    start = epoch_index
    epoch_index += size_of_batch
    if epoch_index > examples_number:
        start = 0
        epoch_index = size_of_batch
    end = epoch_index
    
    return data[start:end], labels[start:end]

# Training Loop

In [64]:
for epoch in range(epoch_number):
    for i in range(iteration_number):
        batch_x, batch_y = get_batch(size_of_batch, x_train, y_train)
        feed_dictionary = {X:batch_x, Y:batch_y}
        # running optimizer, running calcalation on given data
        session.run(train_step, feed_dict=feed_dictionary)

        s, batch_accuracy = session.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    train_writer.add_summary(s, epoch)
    print(f'Epoch : {epoch} \t| Training Accuracy : {batch_accuracy.round(3)}')
    
    # validation
    summary = session.run(fetches=merged_summary, feed_dict={X:x_validation, Y:y_validation})
    validation_writer.add_summary(summary, epoch)
print('Done training')

Epoch : 0 	| Training Accuracy : 0.46399998664855957
Epoch : 1 	| Training Accuracy : 0.6589999794960022
Epoch : 2 	| Training Accuracy : 0.7300000190734863
Epoch : 3 	| Training Accuracy : 0.746999979019165
Epoch : 4 	| Training Accuracy : 0.753000020980835
Epoch : 5 	| Training Accuracy : 0.7580000162124634
Epoch : 6 	| Training Accuracy : 0.7580000162124634
Epoch : 7 	| Training Accuracy : 0.7590000033378601
Epoch : 8 	| Training Accuracy : 0.7749999761581421
Epoch : 9 	| Training Accuracy : 0.8510000109672546
Done training


# Reset for the Next Run

In [65]:
train_writer.close()
validation_writer.close()
session.close()
tf.reset_default_graph()